In [2]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore  
from pinecone import Pinecone, ServerlessSpec

# 1. 환경 변수 로딩 (.env 파일에 OPENAI_API_KEY, PINECONE_API_KEY 저장되어 있어야 함)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX = "legal-docs-index"

# 2. Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 3. 인덱스 새로 만들기 (기존에 있으면 삭제 후 생성)
if PINECONE_INDEX in [index.name for index in pc.list_indexes()]:
    pc.delete_index(PINECONE_INDEX)

pc.create_index(
    name=PINECONE_INDEX,
    dimension=1536,  # text-embedding-3-small 모델 차원 수
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# 4. PDF 로드 및 전처리
pdf_files = [
    "개인정보 보호법(법률)(제19234호)(20240315).pdf",
    "주택임대차보호법(법률)(제19356호)(20230719).pdf",
    "근로기준법(법률)(제18176호)(20211119).pdf"
]
all_chunks = []
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    chunks = splitter.split_documents(pages)

    # 출처 기록
    for chunk in chunks:
        chunk.metadata["source"] = pdf_path

    all_chunks.extend(chunks)

# 5. 임베딩 모델 (OpenAI)
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY
)

# 6. 벡터스토어에 저장 (최신 방식)
vectorstore = PineconeVectorStore.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    index_name=PINECONE_INDEX,
    pinecone_api_key=PINECONE_API_KEY
)

print(f"✅ PDF 3개를 '{PINECONE_INDEX}' 인덱스로 Pinecone에 성공적으로 저장 완료!")


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15916\1361119951.py:50: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


✅ PDF 3개를 'legal-docs-index' 인덱스로 Pinecone에 성공적으로 저장 완료!


In [3]:
import os
from dotenv import load_dotenv
from typing import Dict, Any
from datetime import datetime
from pydantic import BaseModel
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langgraph.graph import StateGraph, END
from pinecone import Pinecone, ServerlessSpec
from fpdf import FPDF

In [4]:
# 1. 환경변수 및 모델 로딩
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX = "legal-docs-index"

In [5]:
# 2. Pinecone 인덱스 객체 가져오기
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX)

In [6]:
# 3. 임베딩 및 벡터스토어 구성
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

vectorstore = PineconeVectorStore(
    index=index,
    embedding=embedding_model
)

llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15916\1966544794.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)


In [7]:
# 4. 상태 모델 정의
class AgentState(BaseModel):
    query: str
    context: Dict[str, Any] = {}

In [8]:
"""
┌───────────────┐ → ┌─────────────────┐ → ┌──────────────────────┐ → ┌────────────────────┐
│   질문 분석    │   │   문서 검색      │   │   판단 (근거 포함)    │   │ 요약 및 조치 제안   │
│  (HITL 적용)   │   │                 │   │   (HITL 적용)        │   │                    │
└───────────────┘   └─────────────────┘   └──────────────────────┘   └────────────────────┘
"""

'\n┌───────────────┐ → ┌─────────────────┐ → ┌──────────────────────┐ → ┌────────────────────┐\n│   질문 분석    │   │   문서 검색      │   │   판단 (근거 포함)    │   │ 요약 및 조치 제안   │\n│  (HITL 적용)   │   │                 │   │   (HITL 적용)        │   │                    │\n└───────────────┘   └─────────────────┘   └──────────────────────┘   └────────────────────┘\n'

In [9]:
#AI Agent 생성

# 질문 분석 Agent
analyze_prompt = SystemMessage(content="""
당신은 법률 상담 시스템의 질문 분석가입니다.
사용자의 질문에서 법률 분야(예: 민사, 형사, 노동)와 핵심 쟁점을 도출하세요.
""")

def analyze_agent(state: AgentState) -> AgentState:
    messages = [analyze_prompt, {"role": "user", "content": state.query}]
    result = llm.invoke(messages)
    original_analysis = getattr(result, "content", str(result))

    # Human-in-the-loop: 사용자 수동 수정 유도 (질문 분석)
    print("\n[HITL] 질문 분석 결과:")
    print(original_analysis)
    user_input = input("[HITL] 분석 결과를 수정하려면 입력 (Enter: 그대로 사용): ")
    state.context["analysis"] = user_input.strip() if user_input.strip() else original_analysis

    return state

def retrieve_agent(state: AgentState) -> AgentState:
    docs = vectorstore.as_retriever(search_kwargs={"k": 5}).get_relevant_documents(state.query)
    state.context["documents"] = docs
    return state

# 법률 판단 Agent
reason_prompt = SystemMessage(content="""
당신은 법률 전문가입니다. 아래 문서 내용을 바탕으로 질문에 대해 논리적으로 판단하고, 반드시 근거가 포함된 응답을 생성하세요.
""")

def reasoning_agent(state: AgentState) -> AgentState:
    documents = state.context.get("documents", [])
    doc_text = "\n\n".join([d.page_content for d in documents])
    messages = [reason_prompt, {"role": "user", "content": f"질문: {state.query}\n\n문서 내용:\n{doc_text}"}]
    result = llm.invoke(messages)
    original_answer = getattr(result, "content", str(result))

    # 🔧 Human-in-the-loop: 판단 결과 수동 수정
    print("\n[HITL] 판단 결과:")
    print(original_answer)
    user_input = input("[HITL] 판단 결과를 수정하려면 입력 (Enter: 그대로 사용): ")
    state.context["answer"] = user_input.strip() if user_input.strip() else original_answer

    return state

# 요약 및 조치 제공 Agent
summarize_prompt = SystemMessage(content="""
당신은 법률 상담 요약가입니다. 아래 답변을 일반인이 이해하기 쉽도록 정리하고, 적절한 조치 방법을 제안하세요.
""")

def summarize_agent(state: AgentState) -> AgentState:
    base_answer = state.context.get("answer", "")
    messages = [summarize_prompt, {"role": "user", "content": base_answer}]
    result = llm.invoke(messages)
    state.context["summary"] = getattr(result, "content", str(result))
    return state

In [17]:
# 6. LangGraph 워크플로우 구성
workflow = StateGraph(AgentState)
workflow.add_node("analyze", analyze_agent)
workflow.add_node("retrieve", retrieve_agent)
workflow.add_node("reason", reasoning_agent)
workflow.add_node("summarize", summarize_agent)
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "retrieve")
workflow.add_edge("retrieve", "reason")
workflow.add_edge("reason", "summarize")
workflow.add_edge("summarize", END)

In [19]:
# 7. 실행 예시
app = workflow.compile()

query = "임대차 계약 기간 중 임차인이 계약을 해지할 수 있는 조건은 무엇인가요?"
state = AgentState(query=query)
result = app.invoke(state)

ctx = result.get("context", {})

print("\n🔍 질문 분석:\n", ctx.get("analysis"))
print("\n📘 판단 결과:\n", ctx.get("answer"))
print("\n✅ 최종 요약 및 제안:\n", ctx.get("summary"))


[HITL] 질문 분석 결과:
법률 분야: 민사 (임대차법)
핵심 쟁점: 임대차 계약 기간 중 임차인의 계약 해지 조건

[HITL] 판단 결과:
임대차 계약 기간 중 임차인이 계약을 해지할 수 있는 조건은 제6조의2에 따라 계약이 갱신된 경우임을 확인할 수 있습니다. 이 조항에 따르면, 임차인은 언제든지 임대인에게 계약해지를 통지할 수 있습니다. 그러나 이 해지는 임대인이 그 통지를 받은 날부터 3개월이 지나야 그 효력이 발생합니다. 이는 임대인에게 적절한 시간을 제공하여 새로운 임차인을 찾을 수 있도록 하기 위함입니다. 

또한, 임차인이 임차인으로서의 의무를 현저히 위반한 경우에도 계약을 해지할 수 있습니다. 이는 제6조의3에서 임차인이 2기의 차임액에 해당하는 금액에 이르도록 차임을 연체한 사실이 있는 경우, 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경우, 임차인이 임대인의 동의 없이 목적 주택의 전부 또는 일부를 전대한 경우, 임차인이 임차한 주택의 전부 또는 일부를 고의나 중대한 과실로 파손한 경우 등을 예로 들 수 있습니다. 이러한 경우에는 임차인이 계약을 해지할 수 있는 권리를 가집니다.

🔍 질문 분석:
 법률 분야: 민사 (임대차법)
핵심 쟁점: 임대차 계약 기간 중 임차인의 계약 해지 조건

📘 판단 결과:
 임대차 계약 기간 중 임차인이 계약을 해지할 수 있는 조건은 제6조의2에 따라 계약이 갱신된 경우임을 확인할 수 있습니다. 이 조항에 따르면, 임차인은 언제든지 임대인에게 계약해지를 통지할 수 있습니다. 그러나 이 해지는 임대인이 그 통지를 받은 날부터 3개월이 지나야 그 효력이 발생합니다. 이는 임대인에게 적절한 시간을 제공하여 새로운 임차인을 찾을 수 있도록 하기 위함입니다. 

또한, 임차인이 임차인으로서의 의무를 현저히 위반한 경우에도 계약을 해지할 수 있습니다. 이는 제6조의3에서 임차인이 2기의 차임액에 해당하는 금액에 이르도록 차임을 연체한 사실이 있는 경우, 임차인이 거짓이나 그 밖의 부정한 방법으로 임차한 경

In [13]:
# 8. PDF 리포트 생성 (확장 템플릿)
class PDFReport(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)
        self.add_font('Nanum', '', 'NanumGothic.ttf', uni=True)
        self.add_font('Nanum', 'B', 'NanumGothic-Bold.ttf', uni=True) 
        self.set_font('Nanum', '', 12)                       
        self.add_page()                                        

    def header(self):
        self.set_font('Nanum', '', 14)
        self.cell(0, 10, '법률 상담 보고서', ln=True, align='C')
        self.set_font('Nanum', '', 10)
        self.cell(0, 10, f"상담일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True, align='R')
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Nanum', '', 9)
        self.cell(0, 10, f'- {self.page_no()} -', align='C')

    def section(self, title, content):
        self.set_font('Nanum', 'B', 12)
        self.cell(0, 10, title, ln=True)
        self.set_font('Nanum', '', 11)
        self.multi_cell(0, 8, content)
        self.ln()


pdf = PDFReport()
pdf.section("1. 사용자 질문", query)
pdf.section("2. 질문 분석 결과", ctx.get("analysis", "-"))
pdf.section("3. 판단 내용", ctx.get("answer", "-"))
pdf.section("4. 요약 및 조치 제안", ctx.get("summary", "-"))

output_path = "법률_상담_보고서.pdf"
pdf.output(output_path)
print(f"\n📄 PDF 보고서 저장 완료: {output_path}")


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:831: UserWarning: missing glyph 13295
  warnings.warn("missing glyph %s" % (originalGlyphIdx))
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44016
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44019
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44042
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44062
  warnings.warn("cmap value too big/small: %s" % cm)
c:\U


📄 PDF 보고서 저장 완료: 법률_상담_보고서.pdf


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:831: UserWarning: missing glyph 13532
  warnings.warn("missing glyph %s" % (originalGlyphIdx))
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44202
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44277
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -45228
  warnings.warn("cmap value too big/small: %s" % cm)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -45791
  warnings.warn("cmap value too big/small: %s" % cm)
c:\U